In [ ]:
pip install gudhi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 33.9 MB/s eta 0:00:00


In [ ]:
import copy
import gudhi
import scipy
import torch
import random
import string

import numpy as np
import pandas as pd
import networkx as nx
import torch.nn as nn

In [ ]:
def random_id(prefix, length):
    return prefix + ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))

In [ ]:
random_id('X', 6)

'XPQA6PE'

In [ ]:
paper_ids = list(set(random_id('P', 6) for _ in range(5000)))
author_ids = list(set(random_id('A', 6) for _ in range(1000)))

In [ ]:
ssorc = []
for pid in paper_ids:
  authors = random.sample(author_ids, random.randint(1, 3))
  num_cits = random.randint(1, 20)
  paper = {
      'pid': pid,
      'authors': authors,
      '#cits': num_cits
  }
  ssorc.append(paper)

In [ ]:
print(ssorc[0])

{'pid': 'PW8AY6Y', 'authors': ['AJPOD30', 'AB8OYCE'], '#cits': 19}


In [ ]:
pid0 = ssorc[0]['pid']

In [ ]:
edges = [
    (paper['pid'], author)
    for paper in ssorc
    for author in paper['authors']
]
edges = pd.DataFrame(data=edges, columns=['paper','author'])
edges.head()

,paper,author
0,PW8AY6Y,AJPOD30
1,PW8AY6Y,AB8OYCE
2,P6J4HZN,AK6MM8L
3,P6J4HZN,ACWHUSJ
4,P6J4HZN,AKOVQ61


In [ ]:
papers = {paper['pid']: paper['#cits'] for paper in ssorc}
papers = pd.DataFrame.from_dict(data=papers, orient='index', columns=['#cits'])
papers.index.name = 'pid'
papers.head()

,#cits
pid,
PW8AY6Y,19
P6J4HZN,5
PSPKJXI,15
PR55HNL,16
PG22LUR,12


In [ ]:
new_papers = [pid0]
keep_papers = {pid0}

n_papers = 1000

while len(keep_papers) < n_papers:
  new_authors = edges['author'][edges['paper'].isin(new_papers)]
  new_papers = edges['paper'][edges['author'].isin(new_authors)]

  new_papers_set = set(new_papers.values) - keep_papers
  if not new_papers_set:
    break

  keep_papers.update(new_papers_set)
  new_papers = list(new_papers_set)

# generamos una subgrafica usando Breath-First-Search con seed = pid0 y n_papers = 1000
# esta grafica inducira el subcomplejo

In [ ]:
papers = papers.loc[list(keep_papers)]
edges = edges[edges['paper'].isin(papers.index)]

# restringimos papers y edges adecuadamente

In [ ]:
papers.sort_index(inplace=True)
edges.sort_values('paper', inplace=True)
edges.reset_index(drop=True, inplace=True)

# los sorteamos alfabeticamente

In [ ]:
authors = pd.DataFrame(edges['author'].unique(), columns=['author'])
authors.sort_values('author', inplace=True)
authors.head()

,author
388,A0021A4
592,A00AD98
893,A02GC8Q
252,A03ILHX
66,A04Q3B3


In [ ]:
authors.set_index('author', inplace=True, verify_integrity=True)
authors.head()

""
author
A0021A4
A00AD98
A02GC8Q
A03ILHX
A04Q3B3


In [ ]:
authors['aidx'] = np.arange(len(authors))
authors.head()

,aidx
author,
A0021A4,0
A00AD98,1
A02GC8Q,2
A03ILHX,3
A04Q3B3,4


In [ ]:
papers['pidx'] = np.arange(len(papers))
papers.head()

,#cits,pidx
pid,,
P00FXHP,5,0
P01AICT,12,1
P01ZJ82,12,2
P02GGHL,10,3
P02V563,2,4


In [ ]:
edges = edges.join(papers['pidx'], on='paper', how='right')
edges = edges.join(authors['aidx'], on='author', how='right')
edges.sort_index(inplace=True)
edges.head()

,paper,author,pidx,aidx
0,P00FXHP,AGXHPBC,0,416
1,P00FXHP,AZKJKKP,0,912
2,P00FXHP,A8V6YN2,0,244
3,P01AICT,AV4Q2GW,1,797
4,P01AICT,AUAKO5B,1,780


In [ ]:
biadjacency = scipy.sparse.coo_matrix(
    (np.ones(len(edges), dtype=bool),
    (edges['pidx'], edges['aidx']))
)

# la funcion coo_matrix() crea una matriz dispersa en formato de coordenadas
# en su version mas general, se ve asi: coo_matrix((data, (row, col)), shape=(m, n)) donde
# data: 1D array de los valores no nulos que apareceran en la matriz
# (row, col): son arrays (ambos de la misma longitud que len(data)) que indican las coordenadas de los elementos de 'data' en la matriz;
#             especificamente, el valor data[i] se colocara en la posicion (row[i], col[i]).
# shape: es un parametro opcional, si no se proporciona, la forma de la matriz se infiere de max(row)+1 y max(column)+1
# por lo tanto, biadjacency es una matriz dispersa con una fila i por cada paper, una columna j por cada autor, y un 1 en la posicion (i, j) si el autor j escribio el paper i

In [ ]:
biadj_lil = biadjacency.tolil()
for i in range(2):
  print(biadj_lil.rows[i])

# la funcion .tolil() convierte una matriz dispersa al formato lil (list of lists)
# en este formato, cada fila de la matriz se representa como una lista que contiene los indices de las columnas donde hay valores no nulos
# es decir, M.tolil.rows[i] es una lista de las posiciones de columna j tal que M[i, j] es distinto de 0
# por lo tanto, biadj_lil.rows[i] es una lista de los indices de los autores que escribieron el paper con indice i

[244, 416, 912]
[780, 797]


In [ ]:
st = gudhi.SimplexTree()
for authors in biadj_lil.rows:
  st.insert(authors)

D = st.dimension()

# la clase gudhi.SimplexTree() crea una estructura de datos que representa un complejo simplicial
# el metodo .insert() añade el argumento como un simplejo, junto con todos sus sub-simplejos (por cerradura)
# por lo tanto, st representa el complejo simplicial generado por los conjuntos de coautores de cada paper
# es decir, cada conjunto de autores que escribio un paper se considera un simplejo, y se incluyen automaticamente todos sus subconjuntos

In [ ]:
pap_cits_by_authors = [dict() for _ in range(D+1)]
for j, authors in enumerate(biadj_lil.rows):
  k = len(authors)
  pap_cits_by_authors[k-1].setdefault(frozenset(authors),[]).append(papers['#cits'].iloc[j])

# cada item de pap_cits_by_authors[d] es un diccionario asociado a los d-simplejos de coautores C con:
# key = frozenset(C)
# value = [#cits(P) : C = {los autores de P}]

In [ ]:
for d in range(D+1):
  print(random.choice(list(pap_cits_by_authors[d].items())))

(frozenset({469}), [np.int64(3), np.int64(16)])
(frozenset({584, 853}), [np.int64(16)])
(frozenset({563, 171, 845}), [np.int64(2)])


In [ ]:
precochain = [dict() for _ in range(D+1)]
for d in range(len(pap_cits_by_authors)-1, -1, -1):
  for simplex, values in pap_cits_by_authors[d].items():
    st_simplex = gudhi.SimplexTree()
    st_simplex.insert(simplex)
    for face, _ in st_simplex.get_skeleton(st_simplex.dimension()):
      face = frozenset(face)
      precochain[len(face)-1].setdefault(face, []).extend(pap_cits_by_authors[d][simplex])

# cada item de precochain[d] es un diccionario asociado a los d-simplejos de coautores C con:
# key = frozenset(C)
# value = [#cits(P) : C subset {los autores de P}]

In [ ]:
for d in range(D+1):
  print(random.choice(list(precochain[d].items())))

(frozenset({109}), [np.int64(13)])
(frozenset({642, 875}), [np.int64(5)])
(frozenset({57, 877, 327}), [np.int64(12)])


In [ ]:
cochain = [dict() for _ in range(D+1)]
for d in range(len(pap_cits_by_authors)-1, -1, -1):
  for simplex, values in pap_cits_by_authors[d].items():
    st_simplex = gudhi.SimplexTree()
    st_simplex.insert(simplex)
    for face, _ in st_simplex.get_skeleton(st_simplex.dimension()):
      face = frozenset(face)
      value = np.array(precochain[len(face)-1][face])
      cochain[len(face)-1][face] = int(np.sum(value))

# cada item de cochain[d] es un diccionario asociado a los d-simplejos de coautores C con:
# key = frozenset(C)
# value = np.sum[#cits(P) : C subset {los autores de P}]

In [ ]:
for d in range(D+1):
  print(random.choice(list(cochain[d].items())))

(frozenset({622}), 31)
(frozenset({123, 756}), 17)
(frozenset({896, 281, 773}), 2)


In [ ]:
simplices = [dict() for _ in range(D+1)]
for simplex, _ in st.get_skeleton(D):
  k = len(simplex)-1
  simplices[k][frozenset(simplex)] = len(simplices[k])

# cada item de simplices[d] es un diccionario asociado a los d-simplejos de coautores C con:
# key = frozenset(C)
# value = un indice unico asignado a C (inyectivo sobre los d-simplejos)

In [ ]:
for d in range(D+1):
  print(random.choice(list(simplices[d].items())))

(frozenset({123}), 123)
(frozenset({267, 159}), 1097)
(frozenset({546, 267, 13}), 36)


In [ ]:
simplices_list = [[None] * len(simplices[d]) for d in range(D+1)]
for d in range(D+1):
  for simplex, idx in simplices[d].items():
    simplices_list[d][idx] = simplex

# simplices_list[d][i] = el d-simplejo que tiene indice i en el diccionario simplices[d]

In [ ]:
percent = 20

subsimplices = [dict() for _ in range(D+1)]
for d in range(D+1):
  d_simp_list = list(simplices[d].keys())
  m = int(np.ceil((len(d_simp_list)/100)*percent))
  random.shuffle(d_simp_list)
  subsimplices_list = d_simp_list[:m]
  for simplex in subsimplices_list:
    subsimplices[d][simplex] = simplices[d][simplex]

# para cada dimension d:
# calculamos el percent% del numero total de d-simplejos
# seleccionamos aleatoriamente esos simplejos sin reemplazo
# construimos subsimplices[d] como un subdiccionario de simplices[d] con solo esos simplejos
# estos seran los simplejos de los cuales no conocemos sus valores

In [ ]:
mask = [list() for _ in range(D+1)]
for d in range(D+1):
  known_simplices = list(set(simplices[d].keys()) - set(subsimplices[d].keys()))
  for simplex in known_simplices:
    mask[d].append(simplices[d][simplex])

# mask[d] es la lista de los indices de los d-simplejos conocidos, es decir, aquellos que NO estan en subsimplices[d]

In [ ]:
median = []
for d in range(D+1):
  known_simplices = list(set(simplices[d].keys()) - set(subsimplices[d].keys()))
  median.append(np.median([cochain[d][simplex] for simplex in known_simplices]))

# median[d] es la mediana de los valores de cochain[d] de los simplejos conocidos
# cabe recalcar que en la implementacion original hay un error: calculan la mediana de todos los valores (sin quitar los subsimplices_simplices)

In [ ]:
damaged_cochain = copy.deepcopy(cochain)
for d in range(D+1):
  for simplex in subsimplices[d].keys():
    damaged_cochain[d][simplex] = median[d]

# damaged_cochain tiene la misma estructura que cochain pero
# reemplazamos los valores de los simplejos en subsimplices por la mediana correspondiente

In [ ]:
cochain_values = []
for d in range(D+1):
  cochain_values_d = []
  for idx in range(len(simplices[d])):
    cochain_values_d.append(cochain[d][simplices_list[d][idx]])
  cochain_values.append(cochain_values_d)

# cochain_values[d][i] = el valor del d-simplejo con indice i (segun el diccionario simplices[d]) en cochain[d]

In [ ]:
cochain_tensors = []
for d in range(D+1):
  cochain_tensor = torch.zeros((1, len(cochain_values[d])), dtype=torch.float, requires_grad=False)
  cochain_tensor[0, :] = torch.tensor(cochain_values[d], dtype=torch.float, requires_grad=False)
  cochain_tensors.append(cochain_tensor)

# cochain_tensors[d] es un tensor de tamaño (1, M_d), donde M_d es el numero de d-simplejos
# cochain_tensors[d][0][i] = cochain_values[d][i]

In [ ]:
damaged_cochain_values = []
for d in range(D+1):
  damaged_cochain_values_d = []
  for idx in range(len(simplices[d])):
    damaged_cochain_values_d.append(damaged_cochain[d][simplices_list[d][idx]])
  damaged_cochain_values.append(damaged_cochain_values_d)

# damaged_cochain_values[d][i] = el valor del d-simplejo con indice i (segun el diccionario simplices[d]) en damaged_cochain[d]

In [ ]:
damaged_cochain_tensors = []
for d in range(D+1):
  damaged_cochain_tensor = torch.zeros((1, len(damaged_cochain_values[d])), dtype=torch.float, requires_grad=False)
  damaged_cochain_tensor[0, :] = torch.tensor(damaged_cochain_values[d], dtype=torch.float, requires_grad=False)
  damaged_cochain_tensors.append(damaged_cochain_tensor)

# damaged_cochain_tensors[d] es un tensor de tamaño (1, M_d), donde M_d es el numero de d-simplejos
# damaged_cochain_tensors[d][0][i] = damaged_cochain_values[d][i]

In [ ]:
boundaries = []
for d in range(1, D+1):
  values, idx_simplices, idx_faces = [], [], []
  for simplex, idx_simplex in simplices[d].items():
    for i, left_out in enumerate(np.sort(list(simplex))):
      values.append((-1)**i)
      idx_simplices.append(idx_simplex)
      idx_faces.append(simplices[d-1][simplex.difference({left_out})])
  assert len(values) == (d+1) * len(simplices[d])
  boundary = scipy.sparse.coo_matrix(
      (values, (idx_faces, idx_simplices)),
      dtype = np.float32,
      shape = (len(simplices[d-1]), len(simplices[d]))
  )
  boundaries.append(boundary)

# boundaries[d-1] representa la matriz asociada al operador frontera C_d -> C_{d-1}
# esta es una matriz dispersa de tamaño (M_{d-1}, M_d), donde M_d es el numero de d-simplejos

# para cada d-simplejo con indice idx_simplex en simplices[d]:
# obtenemos sus caras de dimension d-1 eliminando un vartice a la vez
# para esto ordenamos los vartices del simplejo con np.sort() y usamos su posicion en la lista ordenada para determinar el signo

# por ejemplo, si simplex = frozenset{9,7,8}, entonces enumerate(np.sort(list(simplex))) = [(0,7), (1,8), (2,9)]
# y se generan las caras {8,9}, {7,9}, {7,8} con signos (-1)^0, (-1)^1, (-1)^2

# se construye la matriz de frontera de nuevo usando .coo_matrix() con:
# values = lista de ±1 que codifican la orientacion de cada cara
# idx_simplices = indice del d-simplejo original (columna en la matriz)
# idx_faces = indice de la cara correspondiente (fila en la matriz)
# de esta manera, boundaries[d][i, j] = ±1 si la i-asima (d-1)-cara es cara del j-asimo d-simplejo

In [ ]:
laplacians = list()
up = scipy.sparse.coo_matrix(boundaries[0] @ boundaries[0].T)
laplacians.append(up)
for d in range(len(boundaries)-1):
  down = boundaries[d].T @ boundaries[d]
  up = boundaries[d+1] @ boundaries[d+1].T
  laplacians.append(scipy.sparse.coo_matrix(down + up))
down = boundaries[-1].T @ boundaries[-1]
laplacians.append(scipy.sparse.coo_matrix(down))


# por definicion, la matriz laplaciana de dimension d esta dada por
# L_d = B_{d+1} B_{d+1}^t + B_d^t B_d donde B_{d+1} = boundaries[d] y B_d = boundaries[d-1]
# recordemos que B_{d+1} B_{d+1}^t es el operador up y B_d^t B_d es el operador down

# para d = 0 no hay operador down, entonces laplacians[0] = boundaries[0] @ boundaries[0].T
# para 0 < d < D, laplacians[d] = boundaries[d+1] @ boundaries[d+1].T + boundaries[d].T @ boundaries[d]
# para d = D no hay operador up, entonces laplacians[D] = boundaries[D-1].T @ boundaries[D-1]
# cada laplacians[d] es una matriz dispersa de tamaño (M_d, M_d), almacenada en formato COO

In [ ]:
for i in range(len(laplacians)):
  L = laplacians[i]
  bigeig = scipy.sparse.linalg.eigsh(L, k=1, which='LM', return_eigenvectors=False)[0]
  laplacians[i] = L * (1.0 / bigeig)

# scipy.sparse.linalg.eigsh(L, k=1, which='LM', return_eigenvectors=False)[0] regresa el eigenvalor mas grande de L
# por lo tanto, este loop normaliza cada una de las matrices laplacianas

In [ ]:
def coo2tensor(A):
  idxs = torch.LongTensor(np.vstack((A.row, A.col)))
  vals = torch.FloatTensor(A.data)
  return torch.sparse_coo_tensor(idxs, vals, size=A.shape, requires_grad=False)

# coo2tensor(A) es un tensor disperso de PyTorch con la misma estructura y tamaño que la matriz dispersa A

In [ ]:
Ls = [coo2tensor(laplacians[d]) for d in range(D+1)]